In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import features


import skimage.feature
from skimage.feature import graycomatrix, graycoprops

In [2]:
#glcm
import itertools
import numpy as np

coords = [[0, 0], [0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]

def comb_idx(a, b, nb_chans):
    return int(a*(nb_chans-1) - (a-1)*a/2 + (b-a -1))

def glcm(patch, ranges, dirs, nb_bins, normalized=False, symmetric=False, sum=False):

    nb_rows = patch.shape[0]
    nb_cols = patch.shape[1]
    try:
        nb_chans = patch.shape[2]
    except:
        nb_chans = 1

    if(sum):
        glcm = np.zeros((len(ranges), nb_chans, nb_bins, nb_bins)).astype(np.int32)

        for dist, dir, px, py, chan_src in itertools.product(range(len(ranges)),
                                                             range(len(dirs)),
                                                             range(nb_rows),
                                                             range(nb_cols),
                                                             range(nb_chans)):
            direction = coords[dirs[dir]] * ranges[dist]

            if nb_rows > px + direction[0] >= 0 and nb_cols > py + direction[1] >= 0:
                try:
                    src = patch[px, py, chan_src]
                    dst = patch[px + direction[0], py + direction[1], chan_src]
                except:
                    src = patch[px, py]
                    dst = patch[px + direction[0], py + direction[1]]

                if (symmetric):
                    glcm[dist, chan_src, dst, src] += 1
                glcm[dist, chan_src, src, dst] += 1

        if (normalized):
            for dist, chan_src in itertools.product(range(len(ranges)),
                                                         range(nb_chans)):
                glcm = glcm.astype(np.float64)
                min_glcm = np.min(glcm[dist, dir, chan_src, :, :])
                max_glcm = np.max(glcm[dist, dir, chan_src, :, :])
                glcm[dist, dir, chan_src, :, :] = (glcm[dist, dir, chan_src, :, :] - min_glcm) / (max_glcm - min_glcm)

                # glcm[dist, chan_src, :, :] /= np.sum(glcm[dist, chan_src, :, :])

        return glcm


    else:
        glcm = np.zeros((len(ranges), len(dirs), nb_chans, nb_bins, nb_bins)).astype(np.int32)

        for dist, dir, px, py, chan_src in itertools.product(range(len(ranges)),
                                                   range(len(dirs)),
                                                   range(nb_rows),
                                                   range(nb_cols),
                                                   range(nb_chans)):
            direction = coords[dirs[dir]] * ranges[dist]

            if nb_rows > px + direction[0] >= 0 and nb_cols > py + direction[1] >= 0:
                try:
                    src = patch[px, py, chan_src]
                    dst = patch[px + direction[0], py + direction[1], chan_src]
                except:
                    src = patch[px, py]
                    dst = patch[px + direction[0], py + direction[1]]

                    if(symmetric):
                        glcm[dist, dir, chan_src, dst, src] += 1
                    glcm[dist, dir, chan_src, src, dst] += 1

        if(normalized):
            for dist, dir, chan_src in itertools.product(range(len(ranges)),
                                                                   range(len(dirs)),
                                                                   range(nb_chans)):
                glcm = glcm.astype(np.float64)
                min_glcm = np.min(glcm[dist, dir, chan_src, :, :])
                max_glcm = np.max(glcm[dist, dir, chan_src, :, :])
                glcm[dist, dir, chan_src, :, :] = (glcm[dist, dir, chan_src, :, :] - min_glcm) / (max_glcm - min_glcm)
                # glcm[dist, dir, chan_src, :, :] /= np.sum(glcm[dist, dir, chan_src, :, :])

        return glcm

In [3]:
# Define the directories
# Store your directories in a "data" folder in your working directory
CURRENT_DIR = os.getcwd()
TRAIN_DIR = CURRENT_DIR + "\\data\\train"
TEST_DIR = CURRENT_DIR + "\\data\\test"

# Create empty DataFrames for the train and test data
train_df = pd.DataFrame()
test_df = pd.DataFrame()


#input is a plt.imread object
#output is a cropped plt.imread object
def crop_image(image_path):
    image = plt.imread(image_path)
    temp = cv2.dilate(cv2.erode(image, np.ones((3,3), np.uint8), iterations=1), np.ones((3,3), np.uint8), iterations=1)
    a = np.sum(temp, axis=0).nonzero()[0]
    b = np.sum(temp, axis=1).nonzero()[0]
    return image[b[0]:b[-1], a[0]:a[-1]]

#input is a plt.imread object
#output is a normalized plt.imread object
def normalize_image(image):
    return (image - np.min(image))/(np.max(image) - np.min(image))

# Define a function to read and resize an image
def read_and_resize_image(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (128, 128))
    # Convert the resized image to grayscale (2D array)
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return gray_image  # Now, the image is a 2D NumPy array (grayscale)

def process_image(DIR):

    #Initiate Dataframe
    df = pd.DataFrame()

    # Create an empty list to store the new rows
    new_rows = []

    # Iterate over the training directory and read in the images
    for subdir in os.listdir(DIR):
        subdir_path = os.path.join(DIR, subdir)
        for image in os.listdir(subdir_path):
            image_path = os.path.join(subdir_path, image)

            # Read and resize the image
            # resized_image = read_and_resize_image(image_path)
            resized_image = crop_image(image_path)

            # Create a new row for the train DataFrame
            new_row = {
                "image": resized_image,
                "label": {
                    "NonDemented": 0,
                    "VeryMildDemented": 1,
                    "MildDemented": 2,
                    "ModerateDemented": 3
                }[subdir]
            }

            # Add the new row to the list
            new_rows.append(new_row)

    # Concatenate the new rows to the train DataFrame
    df = pd.DataFrame(new_rows)
    
    return df

# Process train and test DataFrames
train_df = process_image(TRAIN_DIR)
test_df = process_image(TEST_DIR)

In [4]:
def compute_mean(img):
    return np.mean(img)

train_mean_df = pd.DataFrame()
test_mean_df = pd.DataFrame()

train_mean_df['mean'] = train_df['image'].apply(compute_mean)
test_mean_df['mean'] = test_df['image'].apply(compute_mean)

In [7]:
from skimage.measure import shannon_entropy
# Compute Entropy
# input image
# output entropy: value (float)
def compute_entropy(img):
    return shannon_entropy(img)

train_entropy_df = pd.DataFrame()
test_entropy_df = pd.DataFrame()

train_entropy_df['entropy'] = train_df['image'].apply(compute_entropy)
test_entropy_df['entropy'] = test_df['image'].apply(compute_entropy)

In [ ]:
def compute_n_moment(img, n, pearson = True):
    flat_img = img.flatten()
    avg = np.mean(flat_img)
    std_dev = np.std(flat_img)
    total = 0
    for i in range(flat_img.shape[0]):
        total += ((flat_img[i] - avg)**n)/((flat_img.shape[0] - 1)*std_dev**n)

    return total - 3 if (not pearson and n == 4) else total

train_skew_df = pd.DataFrame()
test_skew_df = pd.DataFrame()
train_kurtosis_df = pd.DataFrame()
test_kurtosis_df = pd.DataFrame()

train_skew_df['skew'] = train_df['image'].apply(lambda x: compute_n_moment(x, 3))
test_skew_df['skew'] = test_df['image'].apply(lambda x: compute_n_moment(x, 3))
train_kurtosis_df['kurtosis'] = train_df['image'].apply(lambda x: compute_n_moment(x, 4))
test_kurtosis_df['kurtosis'] = test_df['image'].apply(lambda x: compute_n_moment(x, 4))


In [10]:
def compute_xy_n_moment(img, n, pearson = True):
    h,w = np.shape(img)

    img_sum = np.sum(img)
    x = range(w)
    y = range(h)

    X,Y = np.meshgrid(x,y)

    #Centroid (mean)
    cx = np.sum(img*X)/img_sum
    cy = np.sum(img*Y)/img_sum

    ###Standard deviation
    x2 = (range(w) - cx)**2
    y2 = (range(h) - cy)**2

    X2,Y2 = np.meshgrid(x2,y2)

    #Find the variance
    vx = np.sum(img*X2)/img_sum
    vy = np.sum(img*Y2)/img_sum

    #SD is the sqrt of the variance
    sx,sy = np.sqrt(vx),np.sqrt(vy)

    ### n moment
    xn = (range(w) - cx)**n
    yn = (range(h) - cy)**n

    X3,Y3 = np.meshgrid(xn,yn)

    #Find the nth central moment
    mnx = np.sum(img*X3)/img_sum
    mny = np.sum(img*Y3)/img_sum
    #Skewness is the third central moment divided by SD cubed
    nx = mnx/sx**n
    ny = mny/sy**n

    return (nx-3, ny-3) if (not pearson and n == 4) else (nx, ny)

train_x_skew_df = pd.DataFrame()
test_x_skew_df = pd.DataFrame()
train_y_skew_df = pd.DataFrame()
test_y_skew_df = pd.DataFrame()

train_x_kurtosis_df = pd.DataFrame()
test_x_kurtosis_df = pd.DataFrame()
train_y_kurtosis_df = pd.DataFrame()
test_y_kurtosis_df = pd.DataFrame()

train_x_skew_df['x_skew'], train_y_skew_df['y_skew'] = zip(*train_df['image'].apply(lambda x: compute_xy_n_moment(x, 3)))
test_x_skew_df['x_skew'], test_y_skew_df['y_skew'] = zip(*test_df['image'].apply(lambda x: compute_xy_n_moment(x, 3)))
train_x_kurtosis_df['x_kurtosis'], train_y_kurtosis_df['y_kurtosis'] = zip(*train_df['image'].apply(lambda x: compute_xy_n_moment(x, 4)))
test_x_kurtosis_df['x_kurtosis'], test_y_kurtosis_df['y_kurtosis'] = zip(*test_df['image'].apply(lambda x: compute_xy_n_moment(x, 4)))


In [ ]:
dists = [1]
dirs = [1,2,3]

In [133]:
img = train_df['image'][840]
temp_glcm = glcm(img, dists, dirs, 256, True, False, False)
m = features.mean(temp_glcm)
m.shape

(1, 3, 1, 2, 256)

In [174]:
img = train_df['image'][2641]
temp_glcm = glcm(img, dists, dirs, 256, True, False, False)
nb_dim = temp_glcm.shape[-1]
dims = temp_glcm.shape[:-2]

u = features.mean(temp_glcm)
m1 = 0

for index in np.ndindex(dims):
    for px in range(nb_dim):
        m1 += u[index][0][px]
m1


19.99344676690769

In [175]:
## glcm mean
def compute_glcm_mean(img, dists, dirs, normalise = True, symmetric = False, sum = False):
    temp_glcm = glcm(img, dists, dirs, 256, normalise, symmetric, sum)
    nb_dim = temp_glcm.shape[-1]
    dims = temp_glcm.shape[:-2]

    u = features.mean(temp_glcm)
    m = 0

    for index in np.ndindex(dims):
        for px in range(nb_dim):
            m += u[index][0][px]
    return m

train_glcm_mean_df = pd.DataFrame()
test_glcm_mean_df = pd.DataFrame()

train_glcm_mean_df['glcm_mean'] = train_df['image'].apply(lambda x: compute_glcm_mean(x, dists, dirs))
test_glcm_mean_df['glcm_mean'] = test_df['image'].apply(lambda x: compute_glcm_mean(x, dists, dirs))

gsus 40 mins for mean,, 

In [177]:
test_glcm_mean_df

,glcm_mean
0,18.613866
1,18.477282
2,19.881461
3,19.045957
4,19.086247
...,...
1025,21.070858
1026,20.613695
1027,19.955627
1028,19.689847


In [ ]:
## shade and prominence
def compute_sp(image, dists, dirs, normalise = True, symmetric = False, sum = False):
    sp_list = []
    for img in image:
        img_sp = []
        temp_glcm = glcm(img, dists, dirs, 256, normalise, symmetric, sum)
        shade = features.cluster_shade(temp_glcm).reshape(1,-1)[0]
        prom = features.cluster_prom(temp_glcm).reshape(1,-1)[0]
        for i in shade:
            img_sp.append(i)
        for j in prom:
            img_sp.append(j)
        sp_list.append(img_sp)
    return sp_list

a = [f"shd_dist{dist}_dir{dir}" for dist in dists for dir in dirs]
a.extend(list(f"prom_dist{dist}_dir{dir}" for dist in dists for dir in dirs))

train_sp_df = pd.DataFrame(compute_sp(train_df['image'], dists, dir), columns=a)
test_sp_df = pd.DataFrame(compute_sp(test_df['image'], dists, dir), columns=a)

In [113]:
# Get the image data from the DataFrames
train_images = train_df['image'].values
test_images = test_df['image'].values

# Define the function to compute GLCM features for multiple distances and angles
def compute_glcm_features(images, distances, angles, properties):
    glcm_features = []
    for image in images:
        image_glcm_features = []
        for distance in distances:
            for angle in angles:
                # Calculate GLCM for each image, distance, and angle
                glcm = graycomatrix(image, [distance], [angle], levels=256, symmetric=True, normed=True)
                # Compute the specified GLCM properties
                texture_features = [graycoprops(glcm, prop)[0, 0] for prop in properties]
                image_glcm_features.extend(texture_features)
        # Append the computed features for all distances and angles to the result list
        glcm_features.append(image_glcm_features)
    return glcm_features

# Define the GLCM properties you want to compute
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# Define the distances and angles
distances = [1, 2, 3]
angles = [0, np.pi/4, np.pi/2]

# Compute GLCM features for train and test images
train_glcm_features = compute_glcm_features(train_images, distances, angles, properties)
test_glcm_features = compute_glcm_features(test_images, distances, angles, properties)

# Create DataFrames for train and test features
X_train = pd.DataFrame(train_glcm_features, columns=[f"{prop}_d{distance}_a{angle}" for prop in properties for distance in distances for angle in angles])
X_test = pd.DataFrame(test_glcm_features, columns=[f"{prop}_d{distance}_a{angle}" for prop in properties for distance in distances for angle in angles])


In [115]:
# Extract labels from train and test DataFrames
y_train = train_df["label"]
y_test = test_df["label"]

In [179]:
X_train['entropy'] = train_entropy_df['entropy']
X_test['entropy'] = test_entropy_df['entropy']

In [181]:
X_train['mean'] = train_mean_df['mean']
X_test['mean'] = test_mean_df['mean']

In [185]:
X_train['skew'] = train_skew_df['skew']
X_test['skew'] = test_skew_df['skew']
X_train['kurtosis'] = train_kurtosis_df['kurtosis']
X_test['kurtosis'] = test_kurtosis_df['kurtosis']

In [ ]:
X_train['x_skew'] = train_skew_df['x_skew']
X_test['x_skew'] = test_skew_df['x_skew']
X_train['y_skew'] = train_skew_df['y_skew']
X_test['y_skew'] = test_skew_df['y_skew']
X_train['x_kurtosis'] = train_kurtosis_df['x_kurtosis']
X_test['x_kurtosis'] = test_kurtosis_df['x_kurtosis']
X_train['y_kurtosis'] = train_kurtosis_df['y_kurtosis']
X_test['y_kurtosis'] = test_kurtosis_df['y_kurtosis']

In [187]:
X_train['glcm_mean'] = train_glcm_mean_df['glcm_mean']
X_test['glcm_mean'] = test_glcm_mean_df['glcm_mean']

In [ ]:
X_train = pd.concat([X_train, train_sp_df], axis=1)
X_test = pd.concat([X_test, test_sp_df], axis=1)

In [188]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4129, 61)
(4129,)
(1030, 61)
(1030,)


TODO:
- cropped //
- add mean // 40 mins on my best settings
- nth moment (skewness and kurtosis; n = 3, 4)//
- glcm mean
- glcm variance
- entropy // it's shannon
- cluster shade // but takes kinda long
- cluster prominence // but takes kinda long

In [199]:
directory_path = "processed_data/glcm/"

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Save the train and test DataFrames
X_train.to_pickle("processed_data/glcm/X_train.pkl")
y_train.to_pickle("processed_data/glcm/y_train.pkl")
X_test.to_pickle("processed_data/glcm/X_test.pkl")
y_test.to_pickle("processed_data/glcm/y_test.pkl")